In [ ]:
import glob
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd

def extract_province_data(cmaq_data, province_name, provinces):
    province_of_interest = provinces[provinces['pr_name'] == province_name]
    gdf = province_of_interest.geometry.iloc[0]

    latitudes = cmaq_data.coords['LAT'].values
    longitudes = cmaq_data.coords['LON'].values
    mask = np.zeros(latitudes.shape, dtype=bool)

    for i in range(latitudes.shape[0]):
        for j in range(latitudes.shape[1]):
            point = gpd.points_from_xy([longitudes[i, j]], [latitudes[i, j]])
            if point.within(gdf).any():
                mask[i, j] = True

    mask_da = xr.DataArray(mask, coords=cmaq_data['LAT'].coords, dims=cmaq_data['LAT'].dims)
    extracted_data = cmaq_data.where(mask_da, drop=True)

    return extracted_data

# Load shapefile for provinces
provinces = gpd.read_file('....../dataset/shapefile/Province/province.shp')

# Extract grid data
simu_grid = xr.open_dataset('....../cmaq533-o3_2023-2019Marine/data/mcip/2021/202106/2021060112/27km/GRIDCRO2D_27km.20210603.nc')
latitudes = simu_grid.LAT[0, 0, :, :]
longitudes = simu_grid.LON[0, 0, :, :]

# Open multiple NetCDF files
file_pattern = glob.glob('....../CMAQ-DATA/cmaq-D1-WRF41-202106-KZMIN-N/COMBINE_ACONC_27km_202106*.nc')
nc_pol = xr.open_mfdataset(file_pattern, parallel=True, concat_dim='TSTEP', combine='nested')
date_range = get_datetime(nc_pol['TFLAG'])
cmaq_data = nc_pol.assign_coords({'LAT': latitudes, 'LON': longitudes, 'TIME': date_range})

# List of provinces
provinces_list = ['广东省', '广西壮族自治区', '湖南省', '福建省', '湖北省', '浙江省', '江西省', '江苏省', '贵州省']
province_sim = {'NO2': pd.DataFrame(), 'O3': pd.DataFrame(), 'PM25': pd.DataFrame()}

# Process each province
for province in provinces_list: 
    print(f"Processing simulation data for {province}...")
    extracted_data = extract_province_data(cmaq_data, province, provinces)

    cmaq_province = extracted_data.isel(LAY=slice(0, 1)).mean(dim='LAY').mean(dim=['ROW', 'COL'])
    province_sim['NO2'][province] = cmaq_province.NO2.values
    province_sim['O3'][province] = cmaq_province.O3.values
    province_sim['PM25'][province] = cmaq_province.PM25_TOT.values

    for pollutant in province_sim.keys():
        province_sim[pollutant].index = pd.Index(date_range, name='Time').tz_localize('UTC').tz_convert('Asia/Shanghai').tz_localize(None)

print("Simulation data processed successfully.")